Oi Pessoal! Esse é um projeto que realizamos no bootcamp de data science da EloGroup. Nos ofertaram algumas bases bem grandes sobre COVID no Brasil e sua evolução. A ideia era realizar todo ETL, Análise exploratoria e extrair os insights mais significantes.

# Mapeamento


**Temas propostos:**

Economia - Entender os segmentos de negócio impactados no Estado mais afetado pelo Covid e também nos TOP 5 Municipios mais afetados. #Oportunidades de negócio


**Hipoteses**: {Dono da analise: Resumo}

**Bases**: 
**Covid- 19**: Elo Group |   
**Dados Abertos da receita federal sobre alterações em CNPJ - GOV.BR**: https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj

** Tabela da receita federal com listagem de municipios**: http://200.152.38.155/CNPJ/F.K03200$Z.D10911.MUNICCSV.zip

** Tabela da receita federal com listagem de CNAEs**: http://200.152.38.155/CNPJ/F.K03200$Z.D10911.CNAECSV.zip

## Mindmap

https://miro.com/welcomeonboard/N0Z0cUFUbGxscmNnOHlmeGFwZHpQM1hVb0V0cVJwODBWOE04ZXpkYTh2emxMMTRzdjYwNlV3a2ZkSDc5VHJZV3wzMDc0NDU3MzY1MTg0NTgyODk3?invite_link_id=123999546158



# Bibliotecas

In [ ]:
import pandas as pd
import os
import matplotlib as plt
import numpy as np
import random
import math

# Importação dos dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

df = pd.read_csv("/content/drive/MyDrive/Base - COVID/caso_full.csv")

MessageError: ignored

# Funções úteis

### Obtenção de Trecho Randômico
Ajuda a visualizar trechos diferentes dos dados.

**ATENÇÃO:** Não substitui a análise sistemática dos dados.

In [ ]:
# função para randomizar o ponto de partida de uma amostra da lista
# serve pra gente ver dados variados enquanto trabalha.
# vai que temos algum insight enquanto vemos um trecho de
# dados do meio da base e não os primeiros...
# no pior caso descobrimos o nome louco de algum município.

def randomInterval(lst: list, size: int=10) -> list:
    inicio = random.randint(0, len(lst)-size)
    return lst[inicio:inicio+size]

# randomInterval(df,30) #df só vai ser definida no próximo bloco.
#randomInterval([i for i in range(2000)], 30)

# Desenvolvimento (Base Covid)

## Dados Gerais sobre a *Base*

### Colunas do Dataframe

As colunas que podem ser desnecessárias:
epidemiological_week

As colunas que podem ser redundantes:
estimated_population x estimated_population_2019


In [ ]:
# colunas da tabela
df.info()

### Dados Gerais das Colunas do Dataframe

### Tipos de dados

### Conversão


In [ ]:
categorical = ['city', 'city_ibge_code', 'state', 'place_type', 'state',
               'is_last', 'is_repeated',]
date = ['date', 'last_available_date', ]
numerical = ['epidemiological_week', 'estimated_population','last_available_confirmed',
             'last_available_confirmed_per_100k_inhabitants', 'last_available_death_rate',
             'last_available_deaths', 'order_for_place', 'new_confirmed', 'new_deaths' ]

# todas as colunas da base de dados original
allOriginalColumns = categorical + date + numerical

# id precisa ter o formato string repesentando um inteiro sem ponto.
df['city_ibge_code'] = df['city_ibge_code'].astype('str').str.replace('\.0','')

# estimated_population e estimated_population_2019 estão com formato float
# apesar de aparentemente todos os dados serem inteiros.

# aplicação dos tipos em cada categoria de variavel.
# Sarah disse que não é necessário o mask na linha abaixo. Precisamos analisar.
# Acho que o isnull() é convertido para np.NaN automaticamente, enquanto
# '' é convertido como se houvesse algo, então precisamos tratar esse caso específico.
for col in categorical:
    df[col] = df[col].astype('str').mask(df[col].isnull(), np.NaN)
    
for col in date:
    df[col] = pd.to_datetime(df[col])

#randomInterval (df, 10)
df

### Tipos depois da conversão
Tudo certo. Tipos checados.

In [ ]:
### tipos depois da conversão
df.info()

### Describe
Mostra que nossos dados em sua maioria são números de casos e mortes. Pra esses são interessantes os dados de média e dispersão. Pros demais não faz sentido verificar essas variáveis.

In [ ]:
df.describe()

### Número de cidades

Há na base dados de 5298 cidades distintas. O Brasil possui, de acordo com a Wikipedia, 5.570 municípios.

Temos alguma informação de 95% deles.

In [ ]:
df['city'].value_counts()

### Número de cidades que registraram pelo menos 100 casos (conferência)
5.214 cidades das 5.298 que temos registro (98,4%) chegaram a registrar pelo menos 100 casos.

In [ ]:
#display(randomInterval(df['city'].value_counts(),20).sort_values())
(df[df['last_available_confirmed']>=100])['city'].value_counts()
#df['city'].value_counts().describe()

## Remoção de duplicatas
Essa base não tem tuplas repetidas. Entretanto há um campo 'is_duplicated' que tem valor True quando os dados são de uma data passada e já existe uma linha anterior com os mesmos dados.

In [ ]:
print(f'O Dataset possui {len(df.columns)} colunas e {len(df)} entradas')

df.drop_duplicates(inplace=True)
print(f'Removendo as linhas duplicadas ficamos com {len(df)} entradas no total')

## Limpeza de nulos
Os valores nulos na coluna cidade das linhas que representam os dados de estados são previstos.
Os demais precisam ser verificados.

In [ ]:
df.isnull().sum()

### Remoção das linhas com 'city' ==  'Importados/Indefinidos'

Essas linhas não possuem informações relevantes. Foram descobertas através do campo 'estimated_population' nulo.

In [ ]:
randomInterval(df.loc[df['estimated_population'].isnull()])

In [ ]:
df = df.loc[df['city']!='Importados/Indefinidos']

### Amostra de linhas com valores nulos

Esse resultado nos mostra que quando 'estimated_population' e 'estimated_population_2019' são null, não há informação na tabela e o valor de 'city' é a string 'Importados/Indefinidos'.

Precisamos excluir essas linhas.

Observamos que os nulls da coluna que mais tem nulos é inferior a 1% da base. Devemos verificar se essas lacunas têm ou não algum padrão.

### Nulos que ainda sobraram


In [ ]:
df.isnull().sum()

### last_available_confirmed_per_100k_inhabitants


####TODO Zerar a coluna last_avalaible_count_per_100k_inhabitants onde o valor de last_avalaible_confirmed é zero. Ou excluir as linhas com essa informação, o que eu acho justo, dado que não é uma informação relevante para a abertura ou fechamento de CNPJs.

In [ ]:
#df[df['last_available_confirmed_per_100k_inhabitants'].isnull()]
df[def['last_available_confirmed'].isnull()]

In [ ]:
l = np.nan
math.isnan(l)

## Exclusão de 'date'

A coluna 'Epidemiological_week' é composta pelo número do ano concatenado com o número da semana da data e 'date' é uma data futura ou igual à data dos últimos dados coletados antes do registro. De forma que ambas são menos informativas que a 'last_available_date' e serão eliminadas, sendo a 'last_available_date' renomeada para 'date' para ser usada com mais naturalidade.

**Conclusão:** Remoção das duas colunas e renomeação para simplesmente 'date'

In [ ]:
#pd.to_datetime(df['last_available_date'], unit='s').corr(df['epidemiological_week'])
randomInterval(df[['city', 'state', 'epidemiological_week', 'last_available_date', 'date']], 30)
#randomInterval(df['last_available_date']-df['date']) # as colunas parecem ser iguais
#(df['last_available_date']-df['date']).describe() # Não, as colunas não são iguais.


In [ ]:
# Exclusão de 'date'
# df = df.drop('epidemiological_week', 1)
df['date'] = df['last_available_date']
df = df.drop('last_available_date', 1)
randomInterval(df[['city', 'state', 'date', 'last_available_deaths' ]], 30)

In [ ]:
df['date'] = df['last_available_date']
df = df.drop('last_available_date', 1)

###Histograma de 'last_available_date'
Vamos ver quais as datas em que temos mais informação.

In [ ]:
#df['last_available_date'].value_counts().hist()
#last_date_av_df = df['state', 'last_available_date'].group_by(df['last_available_date'].month)
#last_date_av_df.groupby(last_date_av_df["date"].month).count().plot(kind="bar")
#last_date_av_df.value_counts().hist()
#plt.pyplot.plot(last_date_av_df)
len(last_date_av_df.value_counts())
plt.pyplot.plot([i for i in range(len(last_date_av_df.value_counts()))], last_date_av_df.value_counts())

## Dados dos Estados (município não definido)

In [ ]:
(df[df['city'].isnull()]).to_csv('COVID_estados.csv')
randomInterval(df[df['city'].isnull()], 20)

In [ ]:
#dados das populações estimadas sem valor
randomInterval(df[df['estimated_population'].isnull()], 20)

In [ ]:
df[df['city']=='Ipatinga']
#não é assim que se filtra os resultados de um município.

In [ ]:
df.place_type.value_counts()

### Coluna Epidemiological_week e year_month
A coluna Epidemiological_week foi removida por trazer informação redundante de data e não estar na granularidade de tempo que decidimos trabalhar. Criamos em seu lugar a coluna year_month.

### Correlação entre as colunas
r tem o quanto as variáveis são correlacionadas linearmente uma da outra. Seu sinal indica se a relação é direta ou inversa.

In [ ]:
#last_available_date = df['last_available_date']
#epidemiological_week = df['epidemiological_week']
#last_available_date.corr(epidemiological_week)
df.corr()

### Campo 'is_repeated'

Esse campo é verdadeiro quando o campo 'last_avalaible_date' é o mesmo de uma linha anterior.

**Conclusão:** 
Sempre que o valor for True, a linha pode ser removida. Quando as linhas forem eliminadas, a coluna também se torna desnecessária, podendo ser eliminada. São 8% das linhas removidas.

In [ ]:
#df['is_repeated'].unique() # Tem True e False
df['is_repeated'].value_counts() # 
#df[(df['is_repeated']==True) & (df['city']=='Ipatinga')] # 89 resutados em 503 de Ipatinga
#df[df['city']=='Ipatinga']
df = df[df['is_repeated']==False]

## Verificar linhas city = NaN


Verificar se filtrar linhas NaN (Que teoricamente represetam o estado) dá o mesmo valor de somar todos os municipios

**Resultado:** Verificamos que as linhas NaN, apesar de representarem o somatorio do estado da linha acima, não refletem o real numero de casos.

### Campo 'isLast'
indica os dados da última data disponível que é da última data (2021-07-27).
Podemos eliminar.

In [ ]:
#Cria tabela com linhas city = NaN

tabela_NAN = df.loc[(df['city'].isnull() ) & (df['is_last'] == 'True')]
tabela_NAN = tabela_NAN.sort_values(by=['last_available_death_rate'], ascending=False) 
tabela_NAN.head()

#randomInterval(tabela_NAN, 10)
#tabela_NAN.describe()
df.drop('is_repeated')

In [ ]:
# Cria tabela somente com resultados de municipios excluindo valores NaN. 
#  Comparamos valores finais e entendemos que os resultados por estado, SP por exemplo, não batem.
#    Ficamos com a tabela de municipios por oferecer uma maior segurança dos dados e também para correlacionarmos os dados com a tabela de alterações em CNPJs.

tabela_municipios = df.loc[(df['city'].notnull() ) & (df['is_last'] == 'True')] #& (df['state'] == 'SP')
#display(tabela_municipios)

qtd_total = tabela_municipios.groupby('state')['last_available_deaths'].sum()
print(qtd_total)

In [ ]:
# Tabela final

tabela_municipios_last_date = df.loc[(df['city'].notnull() ) & (df['is_last'] == 'True')]
tabela_municipios_evolucao_temporal = df.loc[(df['city'].notnull())]

In [ ]:
tabela_municipios_last_date.head()

italicized text# Análise exploratória dos dados

In [ ]:
# tabela filtrada com ultimas datas ------------->  tabela_municipios_last_date
# tabela filtrada para obtermos evoluções temporais -----------> tabela_municipios_evolucao_temporal

In [ ]:
# Periodo do dataframe (data minima e data maxima)

print(' A menor data é: ' + str(tabela_municipios_evolucao_temporal['date'].min()))
print(' A maior data é: ' + str(tabela_municipios_evolucao_temporal['date'].max()))

In [ ]:
# Quantidade total de mortes

qtd_total = tabela_municipios_last_date['last_available_deaths'].sum()

print('A quantidade total de mortes no Brasil para o período avaliado é: ' + str(qtd_total))

In [ ]:
# Groupby por estado

qtd_estado = tabela_municipios_last_date.groupby('state').agg({'last_available_deaths':'sum',
                                                              'estimated_population':'sum',
                                                              }).reset_index()
qtd_estado['Taxa mortalidade'] = qtd_estado['last_available_deaths']/qtd_estado['estimated_population']
qtd_estado['Mortes a cada 1k'] = (qtd_estado['last_available_deaths'] * 1000)/qtd_estado['estimated_population']
qtd_estado = qtd_estado.sort_values(by=['Taxa mortalidade'], ascending=False)
qtd_estado.head(10)

In [ ]:
tabela_MT = tabela_municipios_last_date.loc[(df['state'] == 'MT')]
tabela_MT = tabela_MT.sort_values(by=['last_available_deaths'], ascending=False)
tabela_MT['tx_mortalidade'] = tabela_MT['last_available_deaths']/tabela_MT['estimated_population']
tabela_MT.head()